In [8]:
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr

# Set up location and time point

In [27]:
# some point in North Pacific Ocean
target_longitude = -170.2
target_latitude = 40.4
# how large should the region around the target location be for averaging values
region_padding_degrees = 0.0
# how many days back do we look for averaging values
days_back = 10
# average salinity/temp values in the region (yes/no)
average_location = False

# current time
time_point = datetime.now()
# another point in time
#time_point = datetime(2023, 12, 25)

# Retrieve data from Copernicus API

In [45]:
import copernicusmarine

def format_data_points(current_datetime, days_back = 30):
    current_date_str = current_datetime.strftime("%Y-%m-%d")
    earlier_datetime = current_datetime - timedelta(days=days_back)
    earlier_date_str = earlier_datetime.strftime("%Y-%m-%d")
    return earlier_date_str, current_date_str

def copernicus_salinity_temp(target_longitude, region_padding_degrees, time_point, average_location = False):
    start_time, end_time = format_data_points(time_point)
    
    measurement_ds = copernicusmarine.open_dataset(
        dataset_id = "cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m",
        minimum_longitude = target_longitude-region_padding_degrees,
        maximum_longitude = target_longitude+region_padding_degrees,
        minimum_latitude = target_latitude-region_padding_degrees,
        maximum_latitude = target_latitude+region_padding_degrees,
        start_datetime = start_time,
        end_datetime = end_time,
        variables = ["sea_water_salinity"],
        # USER DATA ARE KEPT FOR HISTORY REASONS, CAN BE REPLACED WITH SOMETHING MEANINGFUL
        username='test',
        password='test'
    )
    
    temp_ds = copernicusmarine.open_dataset(
        dataset_id = "cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m",
        minimum_longitude = target_longitude-region_padding_degrees,
        maximum_longitude = target_longitude+region_padding_degrees,
        minimum_latitude = target_latitude-region_padding_degrees,
        maximum_latitude = target_latitude+region_padding_degrees,
        start_datetime = start_time,
        end_datetime = end_time,
        variables = ["sea_water_potential_temperature"],
        # USER DATA ARE KEPT FOR HISTORY REASONS, CAN BE REPLACED WITH SOMETHING MEANINGFUL
        username='test',
        password='test'
    )

    measurement_ds['thetao'] = temp_ds['thetao']

    if average_location:
        measurement_ds = measurement_ds.mean(dim=['time', 'latitude', 'longitude'])
    else:
        measurement_ds = measurement_ds.mean(dim=['time'])

    if output == 'df': return measurement_ds.to_dataframe().reset_index() 
    return measurement_ds

In [33]:
measurement_ds = copernicus_salinity_temp(target_longitude, region_padding_degrees, time_point, average_location = average_location, output = output)
measurement_ds

INFO - 2024-05-16T11:39:23Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-05-16T11:39:23Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-05-16T11:39:23Z - The dataset cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m, version '202211', will be retired on the 2024-07-26T00:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-05-16T11:39:24Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-05-16T11:39:26Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-05-16T11:39:26Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-05-16T11:39:26Z - The dataset cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m, version '202211', will be retired on the 2024-07-26T00:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-05-16T11:39:26Z - Service was not sp

<xarray.Dataset> Size: 608B
Dimensions:    (depth: 50, latitude: 1, longitude: 1)
Coordinates:
  * depth      (depth) float32 200B 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 4B 40.42
  * longitude  (longitude) float32 4B -170.2
Data variables:
    so         (depth, latitude, longitude) float32 200B 33.88 33.88 ... nan
    thetao     (depth, latitude, longitude) float32 200B 11.95 11.94 ... nan

# Compute sound speed

In [40]:
import gsw

#https://www.teos-10.org/pubs/gsw/html/gsw_sound_speed_t_exact.html
#https://teos-10.github.io/GSW-Python/gsw_flat.html
def compute_sound_speed_gsw(measurement_ds):
    potential_temps = measurement_ds['thetao']
    latitudes = measurement_ds['latitude']
    longitudes = measurement_ds['longitude']
    depths = measurement_ds['depth']
    salinities = measurement_ds['so']
    pressures = gsw.p_from_z(-depths, latitudes)
    CT_temps = gsw.CT_from_pt(salinities, potential_temps)
    temps = gsw.t_from_CT(salinities, potential_temps, pressures)
    absolute_salinities = gsw.SA_from_SP(salinities, pressures, longitudes, latitudes)
    measurement_ds['temp'] = temps
    measurement_ds['GSW_sound_speed'] = gsw.sound_speed_t_exact(absolute_salinities, temps, pressures)
    return measurement_ds

def Mackenzie_formula(temp, so, depth):
    return 1448.96 + 4.591 * temp - 0.05304 * (temp ** 2) + 2.374e-4 * (temp ** 3) + 1.34 * (so - 35) + 0.0163 * depth

def compute_sound_speed_Mackenzie(measurement_ds):
    depths = measurement_ds['depth']
    salinities = measurement_ds['so']
    pressures = gsw.p_from_z(-depths, latitudes)
    CT_temps = gsw.CT_from_pt(salinities, potential_temps)
    temps = gsw.t_from_CT(salinities, potential_temps, pressures)
    measurement_ds['temp'] = temps
    measurement_ds['Mackenzie_sound_speed'] = Mackenzie_formula(temps, salinities, depths)
    return measurement_ds

In [41]:
compute_sound_speed_Mackenzie(measurement_ds)

<xarray.Dataset> Size: 1kB
Dimensions:                (depth: 50, latitude: 1, longitude: 1)
Coordinates:
  * depth                  (depth) float32 200B 0.494 1.541 ... 5.728e+03
  * latitude               (latitude) float32 4B 40.42
  * longitude              (longitude) float32 4B -170.2
Data variables:
    so                     (depth, latitude, longitude) float32 200B 33.88 .....
    thetao                 (depth, latitude, longitude) float32 200B 11.95 .....
    temp                   (depth, latitude, longitude) float64 400B 11.95 .....
    Mackenzie_sound_speed  (depth, latitude, longitude) float64 400B 1.495e+0...

In [42]:
compute_sound_speed_gsw(measurement_ds)

<xarray.Dataset> Size: 2kB
Dimensions:                (depth: 50, latitude: 1, longitude: 1)
Coordinates:
  * depth                  (depth) float32 200B 0.494 1.541 ... 5.728e+03
  * latitude               (latitude) float32 4B 40.42
  * longitude              (longitude) float32 4B -170.2
Data variables:
    so                     (depth, latitude, longitude) float32 200B 33.88 .....
    thetao                 (depth, latitude, longitude) float32 200B 11.95 .....
    temp                   (depth, latitude, longitude) float64 400B 11.93 .....
    Mackenzie_sound_speed  (depth, latitude, longitude) float64 400B 1.495e+0...
    GSW_sound_speed        (depth, latitude, longitude) float64 400B 1.495e+0...

In [44]:
measurement_ds.to_dataframe().reset_index()

,depth,latitude,longitude,so,thetao,temp,Mackenzie_sound_speed,GSW_sound_speed
0,0.494025,40.416672,-170.166672,33.878208,11.949674,11.933603,1495.158129,1495.239338
1,1.541375,40.416672,-170.166672,33.878456,11.937579,11.921666,1495.134676,1495.215845
2,2.645669,40.416672,-170.166672,33.878883,11.924587,11.908844,1495.109328,1495.190449
3,3.819495,40.416672,-170.166672,33.879299,11.909925,11.894362,1495.079292,1495.160459
4,5.078224,40.416672,-170.166672,33.879375,11.893048,11.877674,1495.042003,1495.123848
5,6.440614,40.416672,-170.166672,33.879295,11.876184,11.861007,1495.005875,1495.088806
6,7.929560,40.416672,-170.166672,33.879196,11.857676,11.842713,1494.966081,1495.050194
7,9.572997,40.416672,-170.166672,33.879150,11.842714,11.827983,1494.941119,1495.026433
8,11.405000,40.416672,-170.166672,33.879154,11.829549,11.815070,1494.925604,1495.012071
9,13.467140,40.416672,-170.166672,33.879166,11.816646,11.802448,1494.914309,1495.002489
